In [15]:
! pip install filterpy
! pip install scikit-image

# Install TensorFlow
! pip install tensorflow

# Install TensorFlow-Object-Detection
! pip install tensorflow-object-detection-api

# Install OpenCV
! pip install opencv-python

# Install Keras (usually comes bundled with TensorFlow)
# Check if Keras is already installed:
! pip show keras

# If not installed, install it:
! pip install keras

     ---------------------------------------- 0.0/577.4 kB ? eta -:--:--
     -- ------------------------------------ 30.7/577.4 kB 1.3 MB/s eta 0:00:01
     -------- ----------------------------- 122.9/577.4 kB 1.2 MB/s eta 0:00:01
     ------------------- ------------------ 297.0/577.4 kB 2.0 MB/s eta 0:00:01
     -------------------------------------  573.4/577.4 kB 3.0 MB/s eta 0:00:01
     -------------------------------------- 577.4/577.4 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/54.5 kB ? eta -:--:--
   ------------------------------------- -- 51.2/54.5 kB ? eta -:--:--
   ---------------------------------------- 54.5/54.5 kB 941.3 kB/s eta 0:00:00
   ---------------------------------------- 0.0/572.7 kB ? eta -:--:--
   ---------------------------------- ---- 512.0/572.7 kB 31.4 MB/s eta 0:00:01
   ---------------------------------------- 572.7/572

In [13]:
! pip install tensorflow-hub

   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   --- ------------------------------------ 0.1/1.7 MB 2.9 MB/s eta 0:00:01
   ------- -------------------------------- 0.3/1.7 MB 3.4 MB/s eta 0:00:01
   -------------- ------------------------- 0.6/1.7 MB 4.2 MB/s eta 0:00:01
   -------------------------- ------------- 1.2/1.7 MB 6.7 MB/s eta 0:00:01
   ---------------------------------------  1.7/1.7 MB 7.8 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 7.9 MB/s eta 0:00:00


In [2]:
"""
    SORT: A Simple, Online and Realtime Tracker
    Copyright (C) 2016-2020 Alex Bewley alex@bewley.ai

    This program is free software: you can redistribute it and/or modify
    it under the terms of the GNU General Public License as published by
    the Free Software Foundation, either version 3 of the License, or
    (at your option) any later version.

    This program is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    GNU General Public License for more details.

    You should have received a copy of the GNU General Public License
    along with this program.  If not, see <http://www.gnu.org/licenses/>.
"""
from __future__ import print_function

import os
import numpy as np
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from skimage import io

import glob
import time
import argparse
from filterpy.kalman import KalmanFilter

np.random.seed(0)


def linear_assignment(cost_matrix):
  try:
    import lap
    _, x, y = lap.lapjv(cost_matrix, extend_cost=True)
    return np.array([[y[i],i] for i in x if i >= 0]) #
  except ImportError:
    from scipy.optimize import linear_sum_assignment
    x, y = linear_sum_assignment(cost_matrix)
    return np.array(list(zip(x, y)))


def iou_batch(bb_test, bb_gt):
  """
  From SORT: Computes IOU between two bboxes in the form [x1,y1,x2,y2]
  """
  bb_gt = np.expand_dims(bb_gt, 0)
  bb_test = np.expand_dims(bb_test, 1)
  
  xx1 = np.maximum(bb_test[..., 0], bb_gt[..., 0])
  yy1 = np.maximum(bb_test[..., 1], bb_gt[..., 1])
  xx2 = np.minimum(bb_test[..., 2], bb_gt[..., 2])
  yy2 = np.minimum(bb_test[..., 3], bb_gt[..., 3])
  w = np.maximum(0., xx2 - xx1)
  h = np.maximum(0., yy2 - yy1)
  wh = w * h
  o = wh / ((bb_test[..., 2] - bb_test[..., 0]) * (bb_test[..., 3] - bb_test[..., 1])                                      
    + (bb_gt[..., 2] - bb_gt[..., 0]) * (bb_gt[..., 3] - bb_gt[..., 1]) - wh)                                              
  return(o)  


def convert_bbox_to_z(bbox):
  """
  Takes a bounding box in the form [x1,y1,x2,y2] and returns z in the form
    [x,y,s,r] where x,y is the centre of the box and s is the scale/area and r is
    the aspect ratio
  """
  w = bbox[2] - bbox[0]
  h = bbox[3] - bbox[1]
  x = bbox[0] + w/2.
  y = bbox[1] + h/2.
  s = w * h    #scale is just area
  r = w / float(h)
  return np.array([x, y, s, r]).reshape((4, 1))


def convert_x_to_bbox(x,score=None):
  """
  Takes a bounding box in the centre form [x,y,s,r] and returns it in the form
    [x1,y1,x2,y2] where x1,y1 is the top left and x2,y2 is the bottom right
  """
  w = np.sqrt(x[2] * x[3])
  h = x[2] / w
  if(score==None):
    return np.array([x[0]-w/2.,x[1]-h/2.,x[0]+w/2.,x[1]+h/2.]).reshape((1,4))
  else:
    return np.array([x[0]-w/2.,x[1]-h/2.,x[0]+w/2.,x[1]+h/2.,score]).reshape((1,5))


class KalmanBoxTracker(object):
  """
  This class represents the internal state of individual tracked objects observed as bbox.
  """
  count = 0
  def __init__(self,bbox):
    """
    Initialises a tracker using initial bounding box.
    """
    #define constant velocity model
    self.kf = KalmanFilter(dim_x=7, dim_z=4) 
    self.kf.F = np.array([[1,0,0,0,1,0,0],[0,1,0,0,0,1,0],[0,0,1,0,0,0,1],[0,0,0,1,0,0,0],  [0,0,0,0,1,0,0],[0,0,0,0,0,1,0],[0,0,0,0,0,0,1]])
    self.kf.H = np.array([[1,0,0,0,0,0,0],[0,1,0,0,0,0,0],[0,0,1,0,0,0,0],[0,0,0,1,0,0,0]])

    self.kf.R[2:,2:] *= 10.
    self.kf.P[4:,4:] *= 1000. #give high uncertainty to the unobservable initial velocities
    self.kf.P *= 10.
    self.kf.Q[-1,-1] *= 0.01
    self.kf.Q[4:,4:] *= 0.01

    self.kf.x[:4] = convert_bbox_to_z(bbox)
    self.time_since_update = 0
    self.id = KalmanBoxTracker.count
    KalmanBoxTracker.count += 1
    self.history = []
    self.hits = 0
    self.hit_streak = 0
    self.age = 0

  def update(self,bbox):
    """
    Updates the state vector with observed bbox.
    """
    self.time_since_update = 0
    self.history = []
    self.hits += 1
    self.hit_streak += 1
    self.kf.update(convert_bbox_to_z(bbox))

  def predict(self):
    """
    Advances the state vector and returns the predicted bounding box estimate.
    """
    if((self.kf.x[6]+self.kf.x[2])<=0):
      self.kf.x[6] *= 0.0
    self.kf.predict()
    self.age += 1
    if(self.time_since_update>0):
      self.hit_streak = 0
    self.time_since_update += 1
    self.history.append(convert_x_to_bbox(self.kf.x))
    return self.history[-1]

  def get_state(self):
    """
    Returns the current bounding box estimate.
    """
    return convert_x_to_bbox(self.kf.x)


def associate_detections_to_trackers(detections,trackers,iou_threshold = 0.3):
  """
  Assigns detections to tracked object (both represented as bounding boxes)

  Returns 3 lists of matches, unmatched_detections and unmatched_trackers
  """
  if(len(trackers)==0):
    return np.empty((0,2),dtype=int), np.arange(len(detections)), np.empty((0,5),dtype=int)

  iou_matrix = iou_batch(detections, trackers)

  if min(iou_matrix.shape) > 0:
    a = (iou_matrix > iou_threshold).astype(np.int32)
    if a.sum(1).max() == 1 and a.sum(0).max() == 1:
        matched_indices = np.stack(np.where(a), axis=1)
    else:
      matched_indices = linear_assignment(-iou_matrix)
  else:
    matched_indices = np.empty(shape=(0,2))

  unmatched_detections = []
  for d, det in enumerate(detections):
    if(d not in matched_indices[:,0]):
      unmatched_detections.append(d)
  unmatched_trackers = []
  for t, trk in enumerate(trackers):
    if(t not in matched_indices[:,1]):
      unmatched_trackers.append(t)

  #filter out matched with low IOU
  matches = []
  for m in matched_indices:
    if(iou_matrix[m[0], m[1]]<iou_threshold):
      unmatched_detections.append(m[0])
      unmatched_trackers.append(m[1])
    else:
      matches.append(m.reshape(1,2))
  if(len(matches)==0):
    matches = np.empty((0,2),dtype=int)
  else:
    matches = np.concatenate(matches,axis=0)

  return matches, np.array(unmatched_detections), np.array(unmatched_trackers)


class Sort(object):
  def __init__(self, max_age=1, min_hits=3, iou_threshold=0.3):
    """
    Sets key parameters for SORT
    """
    self.max_age = max_age
    self.min_hits = min_hits
    self.iou_threshold = iou_threshold
    self.trackers = []
    self.frame_count = 0

  def update(self, dets=np.empty((0, 5))):
    """
    Params:
      dets - a numpy array of detections in the format [[x1,y1,x2,y2,score],[x1,y1,x2,y2,score],...]
    Requires: this method must be called once for each frame even with empty detections (use np.empty((0, 5)) for frames without detections).
    Returns the a similar array, where the last column is the object ID.

    NOTE: The number of objects returned may differ from the number of detections provided.
    """
    self.frame_count += 1
    # get predicted locations from existing trackers.
    trks = np.zeros((len(self.trackers), 5))
    to_del = []
    ret = []
    for t, trk in enumerate(trks):
      pos = self.trackers[t].predict()[0]
      trk[:] = [pos[0], pos[1], pos[2], pos[3], 0]
      if np.any(np.isnan(pos)):
        to_del.append(t)
    trks = np.ma.compress_rows(np.ma.masked_invalid(trks))
    for t in reversed(to_del):
      self.trackers.pop(t)
    matched, unmatched_dets, unmatched_trks = associate_detections_to_trackers(dets,trks, self.iou_threshold)

    # update matched trackers with assigned detections
    for m in matched:
      self.trackers[m[1]].update(dets[m[0], :])

    # create and initialise new trackers for unmatched detections
    for i in unmatched_dets:
        trk = KalmanBoxTracker(dets[i,:])
        self.trackers.append(trk)
    i = len(self.trackers)
    for trk in reversed(self.trackers):
        d = trk.get_state()[0]
        if (trk.time_since_update < 1) and (trk.hit_streak >= self.min_hits or self.frame_count <= self.min_hits):
          ret.append(np.concatenate((d,[trk.id+1])).reshape(1,-1)) # +1 as MOT benchmark requires positive
        i -= 1
        # remove dead tracklet
        if(trk.time_since_update > self.max_age):
          self.trackers.pop(i)
    if(len(ret)>0):
      return np.concatenate(ret)
    return np.empty((0,5))

def parse_args():
    """Parse input arguments."""
    parser = argparse.ArgumentParser(description='SORT demo')
    parser.add_argument('--display', dest='display', help='Display online tracker output (slow) [False]',action='store_true')
    parser.add_argument("--seq_path", help="Path to detections.", type=str, default='data')
    parser.add_argument("--phase", help="Subdirectory in seq_path.", type=str, default='train')
    parser.add_argument("--max_age", 
                        help="Maximum number of frames to keep alive a track without associated detections.", 
                        type=int, default=1)
    parser.add_argument("--min_hits", 
                        help="Minimum number of associated detections before track is initialised.", 
                        type=int, default=3)
    parser.add_argument("--iou_threshold", help="Minimum IOU for match.", type=float, default=0.3)
    args = parser.parse_args()
    return args

In [18]:
import cv2
import tensorflow as tf
from tensorflow.keras.applications import FasterRCNN
from tensorflow.keras.preprocessing import image
import time

# Load pre-trained Faster R-CNN model from TensorFlow
model = FasterRCNN(weights='coco')

# Initialize SORT tracker
tracker = Sort()

# Define target vehicle types
target_vehicles = ["cars", "trucks", "buses"]  # Update this list with desired types

# Labels corresponding to target vehicles in the model's output
target_labels = [model.category_index[v]['id'] for v in target_vehicles]

# Function to perform object detection on a single frame
def detect_objects(frame):
    # Preprocess image for TensorFlow model
    img_tensor = image.img_to_array(frame)
    img_tensor = np.expand_dims(img_tensor, axis=0)
    img_tensor /= 255.0

    # Pass the input through the model
    predictions = model.predict(img_tensor)

    # Extract bounding boxes and labels for target vehicles
    boxes = []
    labels = []
    for box, label, score in zip(predictions[0]['rois'], predictions[0]['class_ids'], predictions[0]['scores']):
        if label in target_labels and score > 0.5:  # Filter by label and confidence score
            boxes.append([box[1], box[0], box[3], box[2]])  # Reshape to [x1, y1, x2, y2] format
            labels.append(label)

    return boxes, labels

# Read video file
video_path = 'traffic_junction.mp4'
cap = cv2.VideoCapture(video_path)

# Get video properties
fps = cap.get(cv2.CAP_PROP_FPS)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define the font and position for displaying frame rate and vehicle count
font = cv2.FONT_HERSHEY_SIMPLEX
org = (10, 40)
font_scale = 1
font_color = (255, 255, 255)
line_type = 2

# Initialize vehicle count dictionary to store counts for each target vehicle type
vehicle_counts = {v: 0 for v in target_vehicles}
total_vehicle_count = 0  # Counter for total distinct vehicles

# Loop through the frames of the video
while cap.isOpened():
    start_time = time.time()  # Record start time for frame processing
    ret, frame = cap.read()
    if not ret:
        break

    # Detect objects in the frame
    boxes, labels = detect_objects(frame)

    # Initialize lists to store detection results for SORT
    detections = []

    # Extract relevant information for each detection and add to the detections list
    for box, label in zip(boxes, labels):
        x1, y1, x2, y2 = box
        detections.append([x1, y1, x2, y2, label])

    # Convert detections to numpy array for tracking
    detections = np.array(detections)

    # Pass detections to SORT tracker
    tracked_objects = tracker.update(detections)

    # Visualize the detections and tracks
    for obj in tracked_objects:
        x1, y1, x2, y2, track_id = obj.astype(int)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Check if the tracked object has a new appearance (not seen before)
        if track_id not in vehicle_counts:
            vehicle_counts[track_id] = 1
            total_vehicle_count += 1
        else:
            vehicle_counts[track_id] += 1  # Increment count for existing tracked object

        # Display track ID and vehicle type (if label matches target)
        if label in target_labels:
            vehicle_type = model.category_index[label]['name']
            cv2.putText(frame, f"{track_id} ({vehicle_type})", (x1, y1 - 5), font, font_scale, font_color, line_type)
        else:
            cv2.putText(frame, str(track_id), (x1, y1 - 5), font, font_scale, font_color, line_type)

    # Calculate frame rate
    end_time = time.time()
    frame_rate = round(1 / (end_time - start_time), 2)

    # Display frame rate and vehicle counts on the frame
    vehicle_count_text = ", ".join([f"{k}: {v}" for k, v in vehicle_counts.items()])
    cv2.putText(frame, 'FPS: {}'.format(frame_rate), org, font, font_scale, font_color, line_type)
    cv2.putText(frame, f"Total Vehicles: {total_vehicle_count}", (10, 80), font, font_scale, font_color, line_type)
    cv2.putText(frame, f"Vehicle Counts: {vehicle_count_text}", (10, 120), font, font_scale, font_color, line_type)

    # Display the frame
    cv2.imshow('Object Tracking', frame)

    # Check for key press to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close OpenCV windows
cap.release()
cv2.destroyAllWindows()

ImportError: cannot import name 'FasterRCNN' from 'tensorflow.keras.applications' (c:\Users\91939\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\_tf_keras\keras\applications\__init__.py)

In [27]:
print(labels_list)

['6', '4', '3', '1', '4', '3', '4', '3', '1', '1', '3', '1', '1', '3', '1', '4', '10', '11', '1', '1', '1', '3', '3', '64', '10', '1', '1', '1', '1', '1', '1', '3', '1', '4', '4', '3', '8', '3', '3', '3', '2', '3', '1', '11', '1', '1', '1', '1', '3', '1', '10', '1', '3', '4', '3', '3', '1', '1', '3', '1', '1', '4', '4', '11', '6', '4', '1', '3', '10', '1', '3', '10', '8', '3', '2', '1', '1', '2', '64', '10', '3', '1', '3', '10', '3', '3', '14', '14', '4', '4', '1', '10', '3', '3', '1', '14', '1', '64', '14', '1', '6', '3', '3', '3', '3', '4', '1', '4', '4', '1', '11', '1', '1', '3', '1', '10', '1', '1', '4', '11', '1', '3', '10', '1', '1', '2', '3', '10', '3', '3', '1', '3', '1', '3', '3', '11', '1', '1', '14', '1', '1', '64', '10', '4', '2', '3', '1', '3', '4', '3', '1', '1', '1', '3', '4', '11', '3', '1', '1', '1', '1', '64', '1', '3', '3', '14', '3', '6', '1', '3', '3', '8', '1', '2', '3', '1', '1', '14', '3', '4', '1', '14', '3', '3', '1', '6', '4', '3', '3', '11', '1', '3', '2', '

In [45]:
model

AttributeError: 'dict' object has no attribute 'summary'

: 

In [ ]:
keys = predictions_list[0].keys()

for key in keys:
    print(key)

print(predictions_list[0]['labels'])
print(predictions_list[0]['scores'])

boxes
labels
scores
tensor([ 6,  4,  3,  1,  4,  3,  4,  3,  1,  1,  3,  1,  1,  3,  1,  4, 10, 11,
         1,  1,  1,  3,  3, 64, 10,  1,  1,  1,  1,  1,  1,  3,  1,  4,  4,  3,
         8,  3,  3,  3,  2,  3,  1, 11,  1,  1,  1,  1,  3,  1, 10,  1,  3,  4,
         3,  3,  1,  1,  3,  1,  1,  4,  4, 11,  6,  4,  1,  3, 10,  1,  3, 10,
         8,  3,  2,  1,  1,  2, 64, 10,  3,  1,  3, 10,  3,  3, 14, 14,  4,  4,
         1, 10,  3,  3,  1, 14,  1, 64, 14,  1])
tensor([0.9937, 0.9434, 0.9267, 0.9103, 0.8593, 0.8509, 0.8296, 0.8262, 0.8223,
        0.7949, 0.7471, 0.6924, 0.6798, 0.6400, 0.5438, 0.5210, 0.4725, 0.4586,
        0.4406, 0.4322, 0.4260, 0.4247, 0.4188, 0.4099, 0.4075, 0.3959, 0.3782,
        0.3743, 0.3651, 0.3557, 0.3439, 0.3428, 0.3297, 0.2953, 0.2951, 0.2926,
        0.2920, 0.2707, 0.2627, 0.2458, 0.2447, 0.2357, 0.2312, 0.2312, 0.2268,
        0.2249, 0.2111, 0.2017, 0.1852, 0.1746, 0.1673, 0.1610, 0.1603, 0.1549,
        0.1539, 0.1452, 0.1446, 0.1443, 0.1416, 0.1